In [20]:
import scipy.io
import numpy as np
import numpy as np
import h5py
import scipy.io
np.random.seed(1337) # for reproducibility
import matplotlib.pyplot as plt

from keras.preprocessing import sequence
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution1D,Convolution2D, MaxPooling1D
from keras.regularizers import l2, activity_l1
from keras.constraints import maxnorm
from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import ModelCheckpoint, EarlyStopping

from keras.optimizers import Adam, SGD



In [36]:

validmat = scipy.io.loadmat('valid4DwStruct.mat')
# testmat = scipy.io.loadmat('test4D.mat')
trainmat = scipy.io.loadmat('train4DwStruct.mat')


X_train = np.transpose(np.array(trainmat['tr'][0][0][0]),axes=(0,2,3,1))
# X_train = np.array(trainmat['tr'][0][0][0])
y_train = np.array(trainmat['tr'][0][0][1]).squeeze()
# y_train = y_train.reshape((-1, 1))
# y_train.squeeze()
print X_train.shape
print y_train.shape



(13736, 2, 8, 60)
(13736,)


In [38]:


lr = 1e-3#learning rate
reg = 1e-3
print 'building model'
nb_filters=32
model = Sequential()
# model.add(LSTM(4,  W_regularizer=l2(reg),return_sequences=True, input_shape=(4, 113)))
# model.add(Activation('relu'))
# model.add(Dropout(0.2))
# model.add(LSTM(64,  W_regularizer=l2(reg),return_sequences=True)) # return sequences is needed for stacking
# model.add(Activation('relu'))
# model.add(Dropout(0.2))
# model.add(LSTM(128,  W_regularizer=l2(reg)))
# model.add(Activation('relu'))
# model.add(Dropout(0.2))
# model.add(Dense(30))
# model.add(Flatten( input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])))

# # model.add(Dense(10))
# # model.add(Dense(1))

# model.add(LSTM(128, 2,29,  W_regularizer=l2(reg),,)) # adding conv layer collapses output
# model.add(Activation('relu'))
# model.add(Dropout(0.2))
model.add(Convolution2D(8, 2,12, border_mode='same', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]))) # adding conv layer collapses output
model.add(Activation('relu'))
model.add(Dropout(0.2)),
model.add(Convolution2D(4, 2,8, border_mode='same', W_regularizer=l2(reg))) # adding conv layer collapses output
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Convolution2D(4, 2,4, border_mode='same', W_regularizer=l2(reg))) # adding conv layer collapses output
model.add(Activation('relu'))
model.add(Dropout(0.2))





model.add(Flatten())


# model.add(Dense(30))
model.add(Dense(1))
adam = Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08)


print 'compiling model'
model.compile(loss='mse', optimizer='adam', metrics=["mse"])



checkpointer = ModelCheckpoint(filepath="bestmodel.hdf5", verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=50, verbose=1)

X_valid=np.transpose(validmat['v'][0][0][0],axes=(0,2,3,1))
y_valid=np.array(validmat['v'][0][0][1]).squeeze()


model.fit(X_train, y_train, batch_size=256, nb_epoch=1000, shuffle=True, validation_data=(X_valid, y_valid),callbacks=[checkpointer,earlystopper])



building model
compiling model
Train on 13736 samples, validate on 723 samples
Epoch 1/1000
13736/13736 [==============================] - 7s - loss: 1.0194 - mean_squared_error: 1.0107 - val_loss: 0.9498 - val_mean_squared_error: 0.9498
Epoch 2/1000
13736/13736 [==============================] - 7s - loss: 1.0080 - mean_squared_error: 0.9998 - val_loss: 0.9406 - val_mean_squared_error: 0.9406
Epoch 3/1000
13736/13736 [==============================] - 7s - loss: 0.9956 - mean_squared_error: 0.9878 - val_loss: 0.9239 - val_mean_squared_error: 0.9239
Epoch 4/1000
 1280/13736 [=>............................] - ETA: 7s - loss: 0.9912 - mean_squared_error: 0.9835

KeyboardInterrupt: 

In [32]:

out=model.predict(X_train, batch_size=512,verbose=1)

model.summary()

13736/13736 [==============================] - 3s     
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_25 (Convolution2D) (None, 8, 8, 60)      392         convolution2d_input_9[0][0]      
____________________________________________________________________________________________________
activation_25 (Activation)       (None, 8, 8, 60)      0           convolution2d_25[0][0]           
____________________________________________________________________________________________________
dropout_25 (Dropout)             (None, 8, 8, 60)      0           activation_25[0][0]              
____________________________________________________________________________________________________
convolution2d_26 (Convolution2D) (None, 4, 8, 60)      516         dropout_25[0][0]                 
____________________________________

In [33]:
%matplotlib
import matplotlib.pyplot as plt

plt.plot(y_train,out,'ro')


Using matplotlib backend: MacOSX


In [27]:
%matplotlib
# X_test=np.transpose(testmat['tt'][0][0][0],axes=(0,2,3,1))

# y_test=np.array(testmat['tt'][0][0][1]).squeeze()

outtest=model.predict(X_valid, batch_size=512,verbose=1)
plt.plot(y_valid,outtest,'ro')

Using matplotlib backend: MacOSX
723/723 [==============================] - 3s     
